In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

In [2]:
cab = pd.read_csv("cab_rides.csv")
weather = pd.read_csv("weather.csv")

In [3]:
cab.head()

,distance,cab_type,time_stamp,destination,source,price,surge_multiplier,id,product_id,name
0,0.44,Lyft,1544952607890,North Station,Haymarket Square,5.0,1.0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,lyft_line,Shared
1,0.44,Lyft,1543284023677,North Station,Haymarket Square,11.0,1.0,4bd23055-6827-41c6-b23b-3c491f24e74d,lyft_premier,Lux
2,0.44,Lyft,1543366822198,North Station,Haymarket Square,7.0,1.0,981a3613-77af-4620-a42a-0c0866077d1e,lyft,Lyft
3,0.44,Lyft,1543553582749,North Station,Haymarket Square,26.0,1.0,c2d88af2-d278-4bfd-a8d0-29ca77cc5512,lyft_luxsuv,Lux Black XL
4,0.44,Lyft,1543463360223,North Station,Haymarket Square,9.0,1.0,e0126e1f-8ca9-4f2e-82b3-50505a09db9a,lyft_plus,Lyft XL


In [4]:
weather.head()

,temp,location,clouds,pressure,rain,time_stamp,humidity,wind
0,42.42,Back Bay,1.0,1012.14,0.1228,1545003901,0.77,11.25
1,42.43,Beacon Hill,1.0,1012.15,0.1846,1545003901,0.76,11.32
2,42.50,Boston University,1.0,1012.15,0.1089,1545003901,0.76,11.07
3,42.11,Fenway,1.0,1012.13,0.0969,1545003901,0.77,11.09
4,43.13,Financial District,1.0,1012.14,0.1786,1545003901,0.75,11.49


In [6]:
def create_dummy(row):
    if row["surge_multiplier"] == 1:
        return 0
    return 1

In [8]:
cab["surge"] = cab.apply(lambda row: create_dummy(row), axis = 1)
cab.head()

,distance,cab_type,time_stamp,destination,source,price,surge_multiplier,id,product_id,name,surge
0,0.44,Lyft,1544952607890,North Station,Haymarket Square,5.0,1.0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,lyft_line,Shared,0
1,0.44,Lyft,1543284023677,North Station,Haymarket Square,11.0,1.0,4bd23055-6827-41c6-b23b-3c491f24e74d,lyft_premier,Lux,0
2,0.44,Lyft,1543366822198,North Station,Haymarket Square,7.0,1.0,981a3613-77af-4620-a42a-0c0866077d1e,lyft,Lyft,0
3,0.44,Lyft,1543553582749,North Station,Haymarket Square,26.0,1.0,c2d88af2-d278-4bfd-a8d0-29ca77cc5512,lyft_luxsuv,Lux Black XL,0
4,0.44,Lyft,1543463360223,North Station,Haymarket Square,9.0,1.0,e0126e1f-8ca9-4f2e-82b3-50505a09db9a,lyft_plus,Lyft XL,0


In [10]:
# convert the datetime fields from epoch to datetime
cab['timestamp'] = pd.to_datetime(cab['time_stamp'], unit='ms')
cab['date_hour'] = cab['timestamp'].dt.strftime('%Y-%m-%d:%H.%M.%S')
weather['timestamp'] = pd.to_datetime(weather['time_stamp'], unit='s')
weather['date_hour'] = weather['timestamp'].dt.strftime('%Y-%m-%d:%H.%M.%S')
    
# drop redundant fields 
cabrides = cab.drop('time_stamp', axis=1)
weather = weather.drop('time_stamp', axis=1)

In [21]:
cab.head()

,distance,cab_type,time_stamp,destination,source,price,surge_multiplier,id,product_id,name,surge,timestamp,date_hour
0,0.44,Lyft,1544952607890,North Station,Haymarket Square,5.0,1.0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,lyft_line,Shared,0,2018-12-16 09:30:07.890,2018-12-16:09.30.07
1,0.44,Lyft,1543284023677,North Station,Haymarket Square,11.0,1.0,4bd23055-6827-41c6-b23b-3c491f24e74d,lyft_premier,Lux,0,2018-11-27 02:00:23.677,2018-11-27:02.00.23
2,0.44,Lyft,1543366822198,North Station,Haymarket Square,7.0,1.0,981a3613-77af-4620-a42a-0c0866077d1e,lyft,Lyft,0,2018-11-28 01:00:22.198,2018-11-28:01.00.22
3,0.44,Lyft,1543553582749,North Station,Haymarket Square,26.0,1.0,c2d88af2-d278-4bfd-a8d0-29ca77cc5512,lyft_luxsuv,Lux Black XL,0,2018-11-30 04:53:02.749,2018-11-30:04.53.02
4,0.44,Lyft,1543463360223,North Station,Haymarket Square,9.0,1.0,e0126e1f-8ca9-4f2e-82b3-50505a09db9a,lyft_plus,Lyft XL,0,2018-11-29 03:49:20.223,2018-11-29:03.49.20


In [22]:
weather.head()

,temp,location,clouds,pressure,rain,humidity,wind,timestamp,date_hour
0,42.42,Back Bay,1.0,1012.14,0.1228,0.77,11.25,2018-12-16 23:45:01,2018-12-16:23.45.01
1,42.43,Beacon Hill,1.0,1012.15,0.1846,0.76,11.32,2018-12-16 23:45:01,2018-12-16:23.45.01
2,42.50,Boston University,1.0,1012.15,0.1089,0.76,11.07,2018-12-16 23:45:01,2018-12-16:23.45.01
3,42.11,Fenway,1.0,1012.13,0.0969,0.77,11.09,2018-12-16 23:45:01,2018-12-16:23.45.01
4,43.13,Financial District,1.0,1012.14,0.1786,0.75,11.49,2018-12-16 23:45:01,2018-12-16:23.45.01


In [11]:
merged = pd.merge(left=cab,
                  right=weather,
                  left_on=['source', 'date_hour'],
                  right_on=['location', 'date_hour'],
                  suffixes=('_ride', '_weather'))

# drop the merge fields -- location = source
drop_cols = ['date_hour', 'location']
merged = merged.drop(drop_cols, axis=1)

In [13]:
merged['rain'] = merged['rain'].fillna(0)

In [14]:
merged.head()

,distance,cab_type,time_stamp,destination,source,price,surge_multiplier,id,product_id,name,surge,timestamp_ride,temp,clouds,pressure,rain,humidity,wind,timestamp_weather
0,0.55,Uber,1543426327706,South Station,Theatre District,7.0,1.0,74b72f4c-28bc-4640-a427-ec60b4b11f82,55c66225-fbe7-4fd5-9072-eab1ece5e23e,UberX,0,2018-11-28 17:32:07.706,41.47,0.85,991.67,0.0,0.63,9.93,2018-11-28 17:32:07
1,1.57,Uber,1543426327658,North End,Theatre District,18.0,1.0,e5aa2477-afb5-4b83-a69d-18ee7d2bfd1d,6c84fd89-3f11-4782-9b50-97c468b19529,Black,0,2018-11-28 17:32:07.658,41.47,0.85,991.67,0.0,0.63,9.93,2018-11-28 17:32:07
2,2.12,Lyft,1543426327658,Northeastern University,Theatre District,10.5,1.0,4d24470f-167a-4d1f-9cab-c123a0713491,lyft,Lyft,0,2018-11-28 17:32:07.658,41.47,0.85,991.67,0.0,0.63,9.93,2018-11-28 17:32:07
3,2.12,Lyft,1543426327658,Northeastern University,Theatre District,7.0,1.0,7902bfaf-5987-499b-9c04-222f4182c581,lyft_line,Shared,0,2018-11-28 17:32:07.658,41.47,0.85,991.67,0.0,0.63,9.93,2018-11-28 17:32:07
4,2.12,Lyft,1543426327658,Northeastern University,Theatre District,16.5,1.0,91129083-ed19-41fd-b16a-06c729ff0790,lyft_plus,Lyft XL,0,2018-11-28 17:32:07.658,41.47,0.85,991.67,0.0,0.63,9.93,2018-11-28 17:32:07


In [19]:
cab.shape

(693071, 13)

In [20]:
merged.shape

(3866, 19)

In [16]:
cab.name.value_counts()

UberXL          55096
WAV             55096
Black SUV       55096
Black           55095
Taxi            55095
UberX           55094
UberPool        55091
Lux             51235
Lyft            51235
Lux Black XL    51235
Lyft XL         51235
Lux Black       51235
Shared          51233
Name: name, dtype: int64

In [17]:
merged.name.value_counts()

Taxi            318
Black SUV       308
Black           307
UberPool        306
UberXL          306
Lyft XL         303
Lyft            302
Lux Black XL    296
WAV             296
Lux Black       296
UberX           293
Lux             270
Shared          265
Name: name, dtype: int64

In [ ]:
def trans_var(row):
    if row["name"].contains("XL"):
        return "XL"
    elif row["name"].contains("XL")